# [Custom Dataset]

* 데이터셋 : irsi.csv
* 커스텀 데이터셋 : IrisDataset

In [13]:
# 모듈로딩

import torch										# tensor 및 기본 처리 함수들
import torch.nn as nn								# 신경망 클래스들 관련
from torch.utils.data import Dataset,DataLoader		# 데이터셋 관련

import pandas as pd									# 데이터 전처리 관련
import numpy as np									# 데이터 로우데이터 관련

In [14]:
DATA_FILE = '../Data/iris.csv'

In [15]:
# 데이터 로딩
irisDF = pd.read_csv(DATA_FILE)
irisDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal.length  150 non-null    float64
 1   sepal.width   150 non-null    float64
 2   petal.length  150 non-null    float64
 3   petal.width   150 non-null    float64
 4   variety       150 non-null    object 
dtypes: float64(4), object(1)
memory usage: 6.0+ KB


# [2] 데이터 전처리

In [16]:
# variety 
irisDF.variety.unique()

array(['Setosa', 'Versicolor', 'Virginica'], dtype=object)

In [17]:
# 'Setosa', 'Versicolor', 'Virginica' ==> 0,1,2 변경
irisDF.variety = irisDF.variety.replace({'Setosa': 0,'Versicolor': 1,'Virginica':2})

print(irisDF.dtypes)

sepal.length    float64
sepal.width     float64
petal.length    float64
petal.width     float64
variety           int64
dtype: object


C:\Users\KDT17\AppData\Local\Temp\ipykernel_24188\3836138713.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  irisDF.variety = irisDF.variety.replace({'Setosa': 0,'Versicolor': 1,'Virginica':2})


# [3] 커스텀 데이터셋 클래스 설계 및 구현

In [18]:
# iris 전용 데이터셋 클래스

class IrisDataset(Dataset):
	# 피쳐와 타겟 분리 및 전처리
	def __init__(self, featureDF, targetDF):
		super().__init__()
		self.feature = featureDF
		self.target = targetDF
		self.rows = featureDF.shape[0]
		self.cols = featureDF.shape[1]
	
	# 데이터셋의 샘플 수 반환 메서드
	def __len__(self):
		return self.rows
	# DataLoader 에서 batch_size 만큼 호출하는 메서드
	# 인덱스에 해당하는  피쳐와 타겟 변환 단, Tensor 형태

	def __getitem__(self, index):
		arrFeature = self.feature.iloc[index].values			# ndarray
		arrTarget = self.target.iloc[index].values				# ndarray
		return torch.FloatTensor(arrFeature), torch.FloatTensor(arrTarget)

In [19]:
# 데이터셋 인스턴스 생성 
featureDF = irisDF[irisDF.columns[:-1]]
targetDF = irisDF[irisDF.columns[-1:]]

print(f'featureDF => {featureDF.shape}, targetDF => {targetDF.shape}')

featureDF => (150, 4), targetDF => (150, 1)


In [34]:
# 확인
dataset = IrisDataset(featureDF, targetDF)
dataset[1]

(tensor([4.9000, 3.0000, 1.4000, 0.2000]), tensor([0.]))

# [4] DataLoader 연동 <hr> 

In [42]:
# DataLoader 인스턴스 생성

loader = DataLoader(dataset, batch_size=3,shuffle=True)
loader


In [41]:
for feature, target in loader:
	print(feature,target, sep='\n')
	break

tensor([[5.1000, 3.5000, 1.4000, 0.2000],
        [4.9000, 3.0000, 1.4000, 0.2000],
        [4.7000, 3.2000, 1.3000, 0.2000]])
tensor([[0.],
        [0.],
        [0.]])
